In [7]:
import os
import torch
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# EDA of the dataset
def eda_vehicle_dataset(dataset):
    print(f"Number of classes: {len(dataset.classes)}")
    print(f"Class names: {dataset.classes}")
    print(f"Number of images: {len(dataset)}")

# Custom Dataset Class for Test Set (No subfolders)
class CustomTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image

# Replace with your actual dataset paths
train_dataset_path = r'E:\Tim\sem 6\DL lab\ex2\q3\data\train'
val_dataset_path = r'E:\Tim\sem 6\DL lab\ex2\q3\data\val'
test_dataset_path = r'E:\Tim\sem 6\DL lab\ex2\q3\data\test'

# Define data transforms
vehicle_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load and explore the datasets
train_dataset = ImageFolder(root=train_dataset_path, transform=vehicle_transform)
val_dataset = ImageFolder(root=val_dataset_path, transform=vehicle_transform)
test_dataset = CustomTestDataset(root_dir=test_dataset_path, transform=vehicle_transform)

eda_vehicle_dataset(train_dataset)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define MLP model for classification with more hidden layers
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLPClassifier, self).__init__()
        layers = []
        for i in range(len(hidden_sizes)):
            if i == 0:
                layers.append(nn.Linear(input_size, hidden_sizes[i]))
            else:
                layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        layers.append(nn.Softmax(dim=1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Instantiate the model, loss function, and optimizer
input_size = 224 * 224 * 3  # Assuming RGB images
hidden_sizes = [128, 64]  # Add more hidden layers as needed
output_size = len(train_dataset.classes)

vehicle_model = MLPClassifier(input_size, hidden_sizes, output_size)
criterion_vehicle = nn.CrossEntropyLoss()
optimizer_vehicle = optim.Adam(vehicle_model.parameters(), lr=0.001)

# Training the model
epochs_vehicle = 10
for epoch in range(epochs_vehicle):
    vehicle_model.train()
    for inputs, labels in train_dataloader:
        inputs = inputs.view(inputs.size(0), -1)  # Flatten the input
        optimizer_vehicle.zero_grad()
        outputs = vehicle_model(inputs)
        loss = criterion_vehicle(outputs, labels)
        loss.backward()
        optimizer_vehicle.step()

    # Validation
    vehicle_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_dataloader:
            val_inputs = val_inputs.view(val_inputs.size(0), -1)
            val_outputs = vehicle_model(val_inputs)
            val_loss += criterion_vehicle(val_outputs, val_labels).item()
            _, predicted = torch.max(val_outputs, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct / total
    print(f'Epoch {epoch + 1}/{epochs_vehicle}, Training Loss: {loss.item()}, Validation Loss: {average_val_loss}, Validation Accuracy: {accuracy}')

# Testing the model
vehicle_model.eval()
test_predictions = []
with torch.no_grad():
    for test_inputs in test_dataloader:
        test_inputs = test_inputs.view(test_inputs.size(0), -1)
        test_outputs = vehicle_model(test_inputs)
        _, test_predicted = torch.max(test_outputs, 1)
        test_predictions.extend(test_predicted.cpu().numpy())

# Save the test predictions if needed
test_predictions_path = 'test_predictions.npy'
np.save(test_predictions_path, np.array(test_predictions))
print(f"Test predictions saved at {test_predictions_path}.")


Number of classes: 10
Class names: ['SUV', 'bus', 'family sedan', 'fire engine', 'heavy truck', 'jeep', 'minibus', 'racing car', 'taxi', 'truck']
Number of images: 1400
Epoch 1/10, Training Loss: 2.3361504077911377, Validation Loss: 2.37186484677451, Validation Accuracy: 0.1
Epoch 2/10, Training Loss: 2.4194839000701904, Validation Loss: 2.37186484677451, Validation Accuracy: 0.1
Epoch 3/10, Training Loss: 2.377817153930664, Validation Loss: 2.37186484677451, Validation Accuracy: 0.1
Epoch 4/10, Training Loss: 2.4194839000701904, Validation Loss: 2.37186484677451, Validation Accuracy: 0.1
Epoch 5/10, Training Loss: 2.3361504077911377, Validation Loss: 2.37186484677451, Validation Accuracy: 0.1
Epoch 6/10, Training Loss: 2.377817153930664, Validation Loss: 2.37186484677451, Validation Accuracy: 0.1
Epoch 7/10, Training Loss: 2.2944839000701904, Validation Loss: 2.37186484677451, Validation Accuracy: 0.1
Epoch 8/10, Training Loss: 2.3361504077911377, Validation Loss: 2.37186484677451, Va